In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers



2025-10-02 15:24:40.616315: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-02 15:24:40.993574: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:

NUM_CLASSES = 10
BATCH_SIZE = 8
LEARNING_RATE = 1e-4


In [ ]:
model = load_model("resnet50_cancer_model-MRI-version-1.keras")

In [ ]:

# Path to your dataset
DATASET_PATH = "./dataset"

# Load training and validation datasets
train_ds = image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(224, 224),
    batch_size=BATCH_SIZE,
    label_mode="categorical"  # Ensures one-hot encoding
)
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.15),
    layers.RandomContrast(0.15),
    layers.RandomTranslation(height_factor=0.05, width_factor=0.05),
])


train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                        num_parallel_calls=tf.data.AUTOTUNE)
val_ds = image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(224, 224),
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Load raw training dataset (no shuffle, no map, no prefetch)
raw_train_ds = image_dataset_from_directory(
    DATASET_PATH,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(224, 224),
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

# Extract labels batch-by-batch (memory-safe)
class_indices = []
for _, label_batch in raw_train_ds:
    class_indices.extend(tf.argmax(label_batch, axis=1).numpy())

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(class_indices),
    y=class_indices
)

class_weights_dict = dict(enumerate(class_weights))
print("Class weights:", class_weights_dict)


In [ ]:
for layer in model.layers[-20:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

# ✅ Recompile with lower learning rate
model.compile(
    optimizer=Adam(learning_rate=5e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
early_exit = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

# ✅ Train
EPOCHS = 15  # Short fine-tuning phase

history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    class_weight=class_weights_dict,
    callbacks=[early_exit, lr_scheduler]
)


In [ ]:
model.save("./models/resnet50_cancer_model-finetuned-version-1.keras")